In [1]:
#-----Torch imports------#
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim


In [2]:
class DataSet():
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, i):
        return self.x[i], self.y[i]

In [3]:
class DataLoader():
    def __init__(self, data, batchSize):
        self.data = data
        self.bs = batchSize
    def __iter__(self):
        for i in range(0, len(self.data.x), self.bs):
            yield(self.data[i:i + self.bs])
        
    

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 10000)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(10000, 784)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out
    
model = Net()
model.double()

Net(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5)
  (fc1): Linear(in_features=3136, out_features=10000, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=10000, out_features=784, bias=True)
)

In [5]:
#Initialize model

In [6]:
learning_rate = .01

In [7]:
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.0 )
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
#def lossFunction(output, target):
#    loss = torch.mean((output - target)**2)
#    return loss

def lossFunction(output, target):
    lossF = nn.BCEWithLogitsLoss()
    loss = lossF(output, target)
    return loss

In [56]:

def train(model, DS, epochs):

    iterations = 0
    cumLoss = 0
    for epoch in range(epochs):
        for xb, yb in DS:

            iterations += 1

            optimizer.zero_grad()

            xb = torch.stack(xb)
            yb = torch.stack(yb)
            

            
            xb = xb.unsqueeze(1)
            
            unitTest(xb,yb)
            
            output = model(xb)

            loss = lossFunction(output, yb)
            loss.backward()
            cumLoss += loss

            optimizer.step()

            optimizer.zero_grad()

            if iterations % 10 == 0:
                print("cum loss is: ", cumLoss / iterations)
                print("loss is: ", loss)
                print(f"Iterations: {iterations}")





In [36]:
import numpy as np

def life_step_1(X):
    """Game of life step using generator expressions"""
    nbrs_count = sum(np.roll(np.roll(X, i, 0), j, 1)
                     for i in (-1, 0, 1) for j in (-1, 0, 1)
                     if (i != 0 or j != 0))
    return (nbrs_count == 3) | (X & (nbrs_count == 2))

def life_step_2(X):
    """Game of life step using scipy tools"""
    from scipy.signal import convolve2d
    nbrs_count = convolve2d(X, np.ones((3, 3)), mode='same', boundary='wrap') - X
    return (nbrs_count == 3) | (X & (nbrs_count == 2))
    
life_step = life_step_1

In [37]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [38]:
def generateRandomConfig():
    X = np.random.random((28, 28))
    X = (X > 0.75)
    return X

In [39]:
import random
def generateData(numberConfigs):
    targetConfig = []
    startConfig = []
    for i in range(0, numberConfigs):
        Y = generateRandomConfig()
        Y = life_step_2(Y)
        Y = life_step_2(Y)
        Y = life_step_2(Y)
        Y = life_step_2(Y)
        X = life_step_2(Y)        
        X = X.astype(double)
        Y = Y.astype(double)
        X = torch.from_numpy(X)
        Y = np.concatenate(Y)
        Y = torch.from_numpy(Y)
        startConfig.append(X)
        targetConfig.append(Y)
 
    return startConfig, targetConfig

In [72]:
dX, dY = generateData(100)

In [73]:
DS = DataSet(dX, dY)

In [74]:
trainData = DataLoader(DS, 10)

In [53]:
for xb, yb in trainData:
    print(xb)
    xb = torch.stack(xb)
    yb = torch.stack(yb)
    print(shape(xb))
    print(shape(yb))
    break

[tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
         1., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
         0., 0., 1., 0., 0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
         0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 1., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 1., 1., 1., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
 

In [75]:
train(model, trainData, 10000)

cum loss is:  tensor(2.0970, dtype=torch.float64, grad_fn=<DivBackward0>)
loss is:  tensor(1.3388, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Iterations: 10


KeyboardInterrupt: 

In [70]:
def unitTest(xb,yb):
    xb = xb.numpy()
    yb = yb.numpy()
    for y,x in zip(yb, xb):
        y = y.astype(bool)
        x = x.astype(bool)
        y = y.reshape(28,28)
        x = x.squeeze(0)
        
        ystepped = life_step_2(y)
        if not numpy.array_equal(x, ystepped):
            print(shape(x), shape(y))
    

In [ ]:
ytest = ytest.numpy()
xtest = xtest.numpy()

In [ ]:
ytest = ytest.reshape(28,28)
xtest = xtest.reshape(28,28)
ytest = ytest.astype(bool)
xtest = xtest.astype(bool)

In [ ]:
print(ytest)

In [ ]:
y_stepped = life_step_2(ytest)

In [ ]:
xtest == y_stepped

In [ ]:
def generateOneConfiguration():

    Y = generateRandomConfig()
    X = Y
    X = life_step_2(X)
    X = life_step_2(X)
    X = life_step_2(X)
    X = life_step_2(X)
    X = life_step_2(X)
    Y = X
    X = life_step_2(X)
    X = X.astype(float)
    X = torch.from_numpy(X)
    Y = Y.astype(float)
    Y = np.concatenate(Y)
    Y = torch.from_numpy(Y)
 
    return X, Y

In [ ]:
X,Y = generateOneConfiguration()

In [ ]:
X = X.unsqueeze(0)
X = X.unsqueeze(0)
pred = model(X)

In [ ]:
print (pred)

In [ ]:
print(lossFunction(pred, Y))

In [ ]:
out = (pred > 0.27).double()

In [ ]:
print(out)

In [ ]:
print(Y)

In [ ]:
loss = torch.sum((out - Y)**2)

In [ ]:
print(loss)

In [ ]:
ys = torch.sum(Y)
print(ys)

In [ ]:
ts = torch.sum(out)
print(ts)

In [ ]:
print(generateRandomConfig())

In [ ]:
dX, dY = generateData(1)

In [ ]:
print (dX)

In [ ]:
   A = life_step_2(dY[0])

In [ ]:
B = dX[0]

In [ ]:
print(A)

In [ ]:
print(B)